# Init for pyqcu.

In [ ]:
import cupy as cp
import numpy as np
from pyqcu import define
from pyqcu import io
from pyqcu import qcu
from pyqcu import eigen, cg
from pyqcu.set import params, argv, set_ptrs
print('My rank is ', define.rank)
params[define._SET_PLAN_] = 1
gauge_filename = f"quda_wilson-bistabcg-gauge_-{params[define._LAT_X_]}-{params[define._LAT_Y_]}-{params  [define._LAT_Z_]}-{params[define._LAT_T_]}-{params[define._LAT_XYZT_]}-{params[define._GRID_X_]}-{params[define._GRID_Y_]}-{params[define._GRID_Z_]}-{params[define._GRID_T_]}-{params[define._PARITY_]}-{params[define._NODE_RANK_]}-{params[define._NODE_SIZE_]}-{params[define._DAGGER_]}-f.h5"
params[define._NODE_RANK_] = define.rank
params[define._NODE_SIZE_] = define.size
qcu.applyInitQcu(set_ptrs, params, argv)
print("Parameters:", params)

# Read from hdf5 files.

In [ ]:
print("Gauge filename:", gauge_filename)
gauge = io.hdf5_xxxtzyx2grid_xxxtzyx(params, gauge_filename)
fermion_in_filename = gauge_filename.replace("gauge", "fermion-in")
print("Fermion in filename:", fermion_in_filename)
fermion_in = io.hdf5_xxxtzyx2grid_xxxtzyx(params, fermion_in_filename)
fermion_out_filename = gauge_filename.replace("gauge", "fermion-out")
print("Fermion out filename:", fermion_out_filename)
quda_fermion_out = io.hdf5_xxxtzyx2grid_xxxtzyx(params, fermion_out_filename)
fermion_out = cp.zeros_like(fermion_in)
print("Fermion out data:", fermion_out.data)
print("Fermion out shape:", fermion_out.shape)
eigenvalues_filename = gauge_filename.replace("gauge", "eigenvalues")
print("Eigenvalues filename:", eigenvalues_filename)
eigenvalues = io.hdf5_xxx2xxx(file_name=eigenvalues_filename)
print("Eigenvalues data:", eigenvalues.data)
print("Eigenvalues shape:", eigenvalues.shape)
eigenvectors_filename = gauge_filename.replace("gauge", "eigenvectors")
print("Eigenvectors filename:", eigenvectors_filename)
eigenvectors = io.eigenvectors2esctzyx(
    params=params, eigenvectors=io.hdf5_xxx2xxx(file_name=eigenvectors_filename))
print("Eigenvectors data:", eigenvectors.data)
print("Eigenvectors shape:", eigenvectors.shape)

# Run wilson bistabcg from pyqcu test.

In [ ]:
qcu.applyWilsonBistabCgQcu(fermion_out, fermion_in, gauge, set_ptrs, params)
print("Fermion out data:", fermion_out.data)
print("Fermion out shape:", fermion_out.shape)
print("QUDA Fermion out data:", quda_fermion_out.data)
print("QUDA Fermion out shape:", quda_fermion_out.shape)
print("Difference:", cp.linalg.norm(fermion_out -
      quda_fermion_out)/cp.linalg.norm(quda_fermion_out))

# Run matvec(eigenvector[.]) ?= eigenvalue[.]*eigenvector[.] for eigen test

In [ ]:
def matvec(src):
    dest = cp.zeros_like(src)
    qcu.applyWilsonCgDslashQcu(
        dest, src, gauge, set_ptrs, params)
    return dest


for i, ev in enumerate(eigenvalues):
    print(f"λ_{i} = {ev:.2e}")
    # Verify eigenvector
    v = eigenvectors[i]
    w = cp.zeros_like(v)
    w = matvec(v)
    error = cp.linalg.norm(w - ev * v) / cp.linalg.norm(w)
    print(f"Relative error: {error:.2e}")
    j = i+1
    if j == len(eigenvalues):
        j = 0
    print(
        f"Diff between λ_{i} and λ_{j}: {cp.linalg.norm(eigenvectors[i] - eigenvectors[j])/cp.linalg.norm(eigenvectors[i]):.2e}")

# Give guage's eigenvalues and eigenvectors to hdf5 files. (pass, don't run this)

In [5]:
# eigen_solver = eigen.solver(
#     n=params[define._LAT_XYZT_] * define._LAT_HALF_SC_, k=define._LAT_Ne_,matvec=matvec,dtype=gauge.dtype)
# eigenvalues, eigenvectors = eigen_solver.run()
# io.xxx2hdf5_xxx(
#     eigenvalues, params, gauge_filename.replace("gauge", "eigenvalues"))
# io.xxx2hdf5_xxx(
#     eigenvectors, params, gauge_filename.replace("gauge", "eigenvectors"))

# MultiGrid - R*vector

In [ ]:
fermion_in = io.xxxtzyx2mg_xxxtzyx(input_array=fermion_in, params=params)
eigenvectors = io.xxxtzyx2mg_xxxtzyx(input_array=eigenvectors, params=params)

# Origin CG

In [ ]:
kappa = 1 / (2 * argv[define._MASS_] + 8)
_params = params.copy()
_params[define._LAT_X_] /= define._LAT_P_
_params[define._LAT_XYZT_] /= define._LAT_P_
_params[define._SET_PLAN_] = 0
_params[define._SET_INDEX_] = 1
qcu.applyInitQcu(set_ptrs, _params, argv)
b_e = fermion_in[define._EVEN_].flatten()
b_o = fermion_in[define._ODD_].flatten()
b__o = cp.zeros_like(b_o)
tmp = cp.zeros_like(b_o)

In [ ]:
# b__o=b_o+kappa*D_oe(b_e)
_params[define._PARITY_] = define._ODD_
qcu.applyWilsonDslashQcu(tmp, b_e, gauge, set_ptrs, _params)
b__o = b_o+kappa*tmp


In [ ]:
# Dslash(x_o)=b__o
cg_solver = cg.slover(b=b__o, matvec=matvec)
x_o = cg_solver.run()


In [ ]:

# x_e  =b_e+kappa*D_eo(x_o)
_params[define._PARITY_] = define._EVEN_
qcu.applyWilsonDslashQcu(tmp, x_o, gauge, set_ptrs, _params)


In [11]:
# give qcu_fermion_out
x_e = b_e+kappa*tmp
qcu_fermion_out = cp.zeros_like(quda_fermion_out)
qcu_fermion_out[define._EVEN_] = x_e.reshape(
    quda_fermion_out[define._EVEN_].shape)
qcu_fermion_out[define._ODD_] = x_o.reshape(
    quda_fermion_out[define._ODD_].shape)

In [ ]:
x_o

In [ ]:
qcu_fermion_out-quda_fermion_out

In [ ]:
quda_fermion_out

# End for pyqcu. (pass, don't run this)

In [15]:
# qcu.applyEndQcu(set_ptrs, params)